### 【 데이터 전용 Dataset/DataLoader 】
- pytorch에서 데이터 관리 및 유지보수를 위한 클래스 제공
- Dataset    : 사용자 데이터에 맞게 커스텀 클래스 생성
- DataLoader : 배치크기만큼 데이터를 추출해 주는 역할
- 데이터셋 분리 => random_split() 함수 제공 : 타겟 클래스 고려하지 않은 랜덤한 데이터 분리

[1] 모듈 로딩 및 데이터 준비 <hr>

In [1]:
## [1-1] 모듈 로딩
import torch                                        # 텐서 및 수치과학 함수들 관련 모듈
from torch.utils.data import Dataset, DataLoader    # pytorch의 데이터 로딩
from torch.utils.data import random_split           # pytorch의 데이터셋 분리 함수
import pandas as pd

In [2]:
## [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

## [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE)
testDF  = pd.read_csv(TEST_FILE)

[2] 커스텀 데이터셋 클래스 생성 및 데이터 적용 <hr>

In [3]:
## -------------------------------------------------------------------------------------
## [2-2] 커스텀 데이터셋 클래스 정의
## -------------------------------------------------------------------------------------
## 클래스이름 : ClfDataset
## 부모클래스 : Dataset
## 오버라이딩 : _ _init_ _(self)         : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
##            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
##            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
##                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
## -------------------------------------------------------------------------------------
class ClfDataset(Dataset):

    ##- 피쳐와 타겟 저장 및 기타 속성 초기화
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    ##- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0]
    
    ##- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.float32)
        return xTS, yTS
    

In [4]:
## -------------------------------------------------------------------------------------
## [2-3] 커스텀 데이터셋 인스턴스 생성 및 사용
## -------------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)   ## <= trainDS, validDS 분리
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 10000,  testDS : 2000


In [5]:
## -------------------------------------------------------------------------------------
## [2-4] 학습용/검증용/테스트용 데이터셋 분리
## -------------------------------------------------------------------------------------
## 학습용   : 순수 학습에 즉, 데이터셋에 규칙/패턴을 찾기 위한 데이터셋
## 검증용   : 제대로 데이터셋에서 규칙/패턴을 찾는지 확인 용도
##           에포크 단위로 찾은 규칙/패턴의 검증용으로 사용
## 테스트용 : 데이터셋에 규칙/패턴 찾은 후 최종 테스트용으로 사용

## 학습용 데이터셋의 개수
print(f'allDS   :{len(allDS)}개, testDS : {len(testDS)}개')

## 학습용 데이터셋 => 학습용:검증용 = 80:20
TRAIN_SIZE = int(0.8 * len(allDS))
VALID_SIZE = len(allDS) - TRAIN_SIZE

print(f'trainDS :{TRAIN_SIZE}개')
print(f'validDS :{VALID_SIZE}개')
print(f'testDS  :{len(testDS)}개')

allDS   :10000개, testDS : 2000개
trainDS :8000개
validDS :2000개
testDS  :2000개


In [6]:
## 학습용 데이터셋 분리 => random_split()
## 단점) 분류의 경우 타겟의 비율 고려되지 않음!
genSeed = torch.Generator().manual_seed(10)
trainDS, validDS = random_split(allDS,
                                [TRAIN_SIZE, VALID_SIZE],
                                generator=genSeed)

print(f'trainDS :{type(trainDS)}, {len(trainDS)}개')
print(f'validDS :{type(validDS)}, {len(validDS)}개')
print(f'testDS  :{type(testDS)}, {len(testDS)}개')

trainDS :<class 'torch.utils.data.dataset.Subset'>, 8000개
validDS :<class 'torch.utils.data.dataset.Subset'>, 2000개
testDS  :<class '__main__.ClfDataset'>, 2000개
